# Custom multi-action agent

This notebook goes through how to create your own custom agent.

An agent consists of two parts:

- Tools: The tools the agent has available to use.
- The agent class itself: this decides which action to take.
        
        
In this notebook we walk through how to create a custom agent that predicts/takes multiple steps at a time.

In [9]:
!pip install langchain

In [10]:
from langchain.agents import AgentExecutor, BaseMultiActionAgent, Tool
from langchain_community.utilities import SerpAPIWrapper

In [11]:
def random_word(query: str) -> str:
    print("\nNow I'm doing this!")
    return "foo"

In [15]:
import os

In [17]:
from google.colab import userdata
SERPAPI_API_KEY=userdata.get('SERPAPI_API_KEY')

In [18]:
os.environ["SERPAPI_API_KEY"] = "da17c0ef2d2f8363db481dace3c36bb05e06a9e4971028ec16bd4c98ede24c0d"

In [20]:
pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=688a3ee1d4cf34b8ba02dd2af71721370e14055a23fe5c5d24324a85ee158acf
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [21]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="RandomWord",
        func=random_word,
        description="call this to get a random word.",
    ),
]

In [32]:
from typing import Any, List, Tuple, Union

from langchain_core.agents import AgentAction, AgentFinish


class FakeAgent(BaseMultiActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

    async def aplan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decided what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        if len(intermediate_steps) == 0:
            return [
                AgentAction(tool="Search", tool_input=kwargs["input"], log=""),
                AgentAction(tool="RandomWord", tool_input=kwargs["input"], log=""),
            ]
        else:
            return AgentFinish(return_values={"output": "bar"}, log="")

In [33]:
agent = FakeAgent()

In [34]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [35]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new AgentExecutor chain...
In 2023, Canada's population jumped by over 1 million people for the first time in the country's history. The population now stands at 39.5 million and is set to pass the 40 million mark later this year.
Now I'm doing this!
foo

> Finished chain.


'bar'

In [26]:
agent_executor.run("How many people live in India as of 2023?")



> Entering new AgentExecutor chain...
1,425,775,850
Now I'm doing this!
foo

> Finished chain.


'bar'

In [29]:
agent_executor.run("what is the capital of india?")



> Entering new AgentExecutor chain...
New Delhi
Now I'm doing this!
foo

> Finished chain.


'bar'

In [28]:
class FakeAgent(BaseMultiActionAgent):
    """Fake Custom Agent."""

    @property
    def input_keys(self):
        return ["input"]

    def plan(
        self, intermediate_steps: List[Tuple[AgentAction, str]], **kwargs: Any
    ) -> Union[List[AgentAction], AgentFinish]:
        """Given input, decide what to do.

        Args:
            intermediate_steps: Steps the LLM has taken to date,
                along with observations.
            **kwargs: User inputs.

        Returns:
            Action specifying what tool to use.
        """
        input_text = kwargs["input"].lower()  # Convert input text to lowercase for case-insensitive matching
        if "it ticket" in input_text:
            return AgentFinish(return_values={"output": "You can raise the IT ticket through the Kuber IT portal."}, log="")
        else:
            return AgentFinish(return_values={"output": "I'm sorry, I couldn't understand your request."}, log="")


In [30]:
agent_executor.run("how to raise IT ticket?")



> Entering new AgentExecutor chain...
['Raise Ticket is commonly used in IT support to request help or report an issue quickly and effectively. Clear communication with customer ...', 'Step 1: To raise a ticket click on “Need Help?” as shown in the below screenshot. ... Step 2: A form opens up fill the required information and click on save ...', 'Contact Cloud Technical Support · Cloud version, this is the site we need to go to to raise ticket (as shared by @Pablito ) · Get your 16-digit ...', '1. Go to the Help & Training Portal. 2. Login using your Salesforce or Marketing Cloud credentials. 3. Click Go on the "My Success Hub" tile. 4.', 'Step 3: Tracking & Analyzing the Tickets · View, track & analyze every support ticket raised. · Select a particular ticket and set the priority as ...', 'A ticket or support request can be created using three options: by the agent or the user on the web portal or by sending an email. Admin can control the ...', "How to create a ticket? Link icon ·

'bar'